In [ ]:
!pip install pyTelegramBotAPI
!pip3 -q install youtube_transcript_api
!pip3 -q install transformers
!pip3 -q install youtube-dl

In [ ]:
####################################################################################
# impor libraries

import os
import re
import psutil
import telebot
import string
import torch
import numpy as np
from datetime import datetime
import youtube_dl



from youtube_transcript_api import YouTubeTranscriptApi
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration, pipeline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
####################################################################################

###################### model for summary
model_name = 'facebook/bart-large-cnn' # can be any model which satisfies quality and inference constraints

tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)#.to(device)

In [ ]:
####################################################################################

###################### tg bot initiation

token = "token"
bot = telebot.TeleBot(token)

###################### predefined data

# we have to predefine the length of output text (currently tokens)
data = {"min_length":100, "max_length":200}

###################### main summarising function

def summary(message, threshold):
    # function that makes summary

    link = message.text
    unique_id = link[threshold:]

    bot.send_message(message.chat.id, "Отлично, это действительно ссылка на YouTuBe видео, дай мне, пожалуйста, немного времени и я скоро отправлю саммари)")

    # count time
    start = datetime.now()

    # make summary part 1) if there are subtitles
    try:
        subtitles_by_intervals = YouTubeTranscriptApi.get_transcript(unique_id, languages=['en', 'ru']) 
        subtitles = " ".join([x['text'] for x in subtitles_by_intervals])
        subtitles = subtitles#.to(device)

        summarizer = pipeline("summarization", model=model_name, tokenizer=model_name, framework='pt') # will be used in audio section as well
        # tokenizer_kwargs = {'truncation': True, 'max_length': 512}

        result = summarizer(
                    subtitles,
                    # tokenizer params
                    # max_length=1024, truncation=True,
                    # model params
                    min_length=data["min_length"],
                    max_length=data["max_length"], 
                    num_beams=2, length_penalty=2.0, early_stopping=True,
                    truncation=True, return_text=True, return_tensors=False
                )[0]['summary_text']

        bot.send_message(message.chat.id, "Саммари готово)")
        bot.send_message(message.chat.id, f"{result}")

        end = datetime.now()
        bot.send_message(message.chat.id, f"Мне потребовалось {np.round((end - start).total_seconds(), 2)} секунд для ответа)")
    
    # if there are no subtitles we can use the approach of downloading audio and represent it as a text
    except:
        bot.send_message(message.chat.id, "Мне надо немного времени, подожди, пожалуйста)")


        ydl_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'wav',
                'preferredquality': '192',
            }],
            'outtmpl':"." + '/video.%(ext)s',
        }

        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([link])

        
        # speech to text model
        model_name_speech_to_text = 'facebook/wav2vec2-base-960h'
        speech_to_text_model = pipeline(model=model_name_speech_to_text)

        absolute_path = "video.wav" # file name of your downloaded audio
        text = speech_to_text_model(absolute_path, chunk_length_s=10) 
        text = text['text'].lower()

        summarizer = pipeline("summarization", model=model_name, tokenizer=model_name, framework='pt') # will be used in audio section as well
        result = summarizer(text, truncation=True)[0]['summary_text']

        bot.send_message(message.chat.id, "Саммари готово)")
        bot.send_message(message.chat.id, f"{result}")

        end = datetime.now()
        bot.send_message(message.chat.id, f"Мне потребовалось {np.round((end - start).total_seconds(), 2)} секунд для ответа)")


##################################### some functions

def num(s):
    # function check that str could be converted to number
    try:
        return int(s)
    except ValueError:
        return "not_a_number"


##################################### command functions

@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, 'Привет!) \nЯ умею делать саммари по видео с YouTuBe. Отправь мне ссылку на видео, и я сделаю по нему саммари)')

    # min and max words in summary
    bot.send_message(message.chat.id, "Только первое, что надо сделать это задать какого размера саммари ты хочешь. \nВведи 2 числа через пробел, где первое - это минимальный размер саммари, а второе - максимальный размер саммари, например:\n100 150")
        

@bot.message_handler(commands=['change_intarval'])
def start_message(message):
    bot.send_message(message.chat.id, 'О, вижу, что появилось желание изменить размер саммари. Схема как и в прошлый раз) \nВведи 2 числа через пробел, где первое - это минимальный размер саммари, а второе - максимальный размер саммари, например:\n250 350')

##################################### text functions

@bot.message_handler(content_types=['text'])
def action_function(message):
    
    # if youtube link has the "brauser" form like https://www.youtube.com/watch?v= we preprocess one way
    if (message.text[:32] == "https://www.youtube.com/watch?v="):
        summary(message, threshold = 32)
        
    # if youtube link has the "iphone" form like https://youtu.be/ we preprocess another way
    elif (message.text[:17] == "https://youtu.be/"):
        summary(message, threshold = 17)

    elif num([number for number in message.text.split(" ")][0]) != "not_a_number":
        list_of_str_numbers = message.text.split(" ")
        list_of_int_numbers = [int(number) for number in list_of_str_numbers]
        
        data["min_length"] = list_of_int_numbers[0]
        data["max_length"] = list_of_int_numbers[1]

        min_length = data["min_length"]
        max_length = data["max_length"]

        bot.send_message(message.chat.id, f"Отлично, получается минимальное число слов, которое ты хочешь - это {min_length}, а максимальное соответственно {max_length}")
        bot.send_message(message.chat.id, f"Теперь я буду по умолчанию считать, что этот интревал ты предпочитаешь по умолчанию, но если хочешь его изменить просто отправь мне /change_intarval")
        bot.send_message(message.chat.id, "Теперь можешь отправлять ссылку на YouTuBe видео)")

    else:
        bot.send_message(message.chat.id, "Проверь, пожалуйста, что ссылка корректна, не могу найти видео на YouTuBe с такой ссылкой)")

#####################################


bot.polling(none_stop=True)